# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import time

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

Using TensorFlow backend.


## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [2]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
#config.display()

## Create Model and Load Trained Weights

In [3]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [4]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']


In [5]:
# Main defining function for blurry image
def blur_image(img, img_background):
    """
    Args:
        img (numpy.array): RGB image. Contains people. Used as "read-only" and will not be modified by this function.
        img_background (numpy.array): RGB image. There are no people in this image. 
    """
    start = time.time()
    
    # img_copy will be modified with the pixel manipulations and returned by the function
    img_copy = np.copy(img)
    
    blur_img=cv2.GaussianBlur(img,(31,31),0)  ##BLUR## 
    
    results = model.detect([img], verbose=0)
    results = results[0]
    
    '''
    ##NOT NEEDED as it shows image with every classifiers
    if display_imgs:
        visualize.display_instances(img, results['rois'], results['masks'], results['class_ids'], 
                                    class_names, results['scores'])
    '''
    # shape of the masks are (h, w, object_ix)
    n_objects_found = len(results['class_ids'])
    for ix in range(n_objects_found):
        if ((results['class_ids'][ix] == (class_names.index('tv'))) 
            or (results['class_ids'][ix] == (class_names.index('person'))) or (results['class_ids'][ix] == (class_names.index('laptop'))) 
            or (results['class_ids'][ix] == (class_names.index('keyboard')))) :      ###if person class is detected
            bbox = results['rois'][ix]
            if bbox is not None:
                x, y, w, h = bbox
                img_face = img[y : y+h, x : x+w]          ###get a different image for person face depending on bbox
                
                p_mask = results['masks'][:, :, ix]       ###get the persons mask

                # The mask is expanded by 10 pixels in all four directions to 
                # ensure the person is entirely covered.
                padding = 10
                p_mask_2 = np.roll(p_mask, padding, axis=0)
                p_mask_3 = np.roll(p_mask, -padding, axis=0)
                p_mask_4 = np.roll(p_mask, padding, axis=1)
                p_mask_5 = np.roll(p_mask, -padding, axis=1)
                p_masks = [p_mask, p_mask_2, p_mask_3, p_mask_4, p_mask_5]
                p_masks_merged = np.logical_or.reduce(p_masks, axis=0)

                other_masks = np.delete(results['masks'], ix, axis=2)
                other_masks_merged = np.logical_or.reduce(other_masks, axis=2)

                person_mask_minus_others = p_masks_merged & np.logical_not(other_masks_merged)

                '''AKIB you need to change here'''

                #img_copy[person_mask_minus_others] = img_background[person_mask_minus_others]   #to make invisible
                img_copy[person_mask_minus_others] = blur_img[person_mask_minus_others]

    
    end = time.time()
    print('{} seconds to process image'.format(int(end - start)))
    
    return img_copy


In [6]:
import cv2

filename=os.path.join(IMAGE_DIR,'type.jpg')
img = skimage.io.imread(filename)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img_background =img
img_processed = blur_image(img, img_background)
cv2.imshow('Final image',img_processed)
cv2.waitKey(0)
cv2.destroyAllWindows()

8 seconds to process image
